# BÁO CÁO: PHÂN TÍCH VÀ XỬ LÝ DỮ LIỆU TITANIC

**Mô tả:**
Báo cáo này thực hiện quy trình khám phá dữ liệu (EDA) và tiền xử lý (Preprocessing) trên tập dữ liệu hành khách Titanic. Các bước bao gồm:
1.  **Khám phá dữ liệu:** Thống kê mô tả, kiểm tra giá trị thiếu.
2.  **Trực quan hóa:** Vẽ biểu đồ Histogram và Boxplot.
3.  **Làm sạch dữ liệu:** Xử lý dữ liệu bị thiếu (Imputation).
4.  **Kỹ thuật đặc trưng:** Chuẩn hóa (Scaling) và Mã hóa (Encoding).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

# Cấu hình hiển thị cho biểu đồ
plt.style.use('seaborn-v0_8-whitegrid')
import warnings
warnings.filterwarnings('ignore') # Tắt cảnh báo để báo cáo sạch đẹp hơn

## 1. Tải dữ liệu và Tổng quan
Dữ liệu được nạp từ file CSV. Chúng ta sẽ xem xét 5 dòng đầu tiên và kích thước tổng thể của tập dữ liệu.

In [ ]:
# Nạp dữ liệu
# Lưu ý: Đảm bảo đường dẫn file chính xác với cấu trúc thư mục của bạn
try:
    df = pd.read_csv("../data/titanic.csv")
    print("Nạp dữ liệu thành công!")
    display(df.head()) # Dùng display() trong notebook đẹp hơn print()
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file dữ liệu. Vui lòng kiểm tra lại đường dẫn.")

# Xác định kích thước
print(f"\nSố dòng dữ liệu: {df.shape[0]}")
print(f"Số cột dữ liệu: {df.shape[1]}")

## 2. Thống kê mô tả và Giá trị thiếu
Bước này giúp phát hiện các bất thường trong dữ liệu và xác định các cột cần được xử lý (điền khuyết).

In [ ]:
# Thống kê mô tả các cột số
print("--- Thống kê mô tả ---")
display(df.describe())

# Đếm giá trị thiếu
print("\n--- Số lượng giá trị thiếu (Null) trên mỗi cột ---")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

# Nhận xét tự động
print("\n=> Nhận xét: Các cột cần xử lý là Age, Cabin và Embarked.")

## 3. Trực quan hóa dữ liệu
Chúng ta sẽ sử dụng hai loại biểu đồ:
* **Histogram:** Để xem phân phối độ tuổi của hành khách.
* **Boxplot:** Để phát hiện các giá trị ngoại lai (outliers) trong giá vé (Fare).



[Image of boxplot anatomy explanation]

In [ ]:
plt.figure(figsize=(12, 5))

# Biểu đồ 1: Histogram cho Age
plt.subplot(1, 2, 1)
sns.histplot(df['Age'].dropna(), bins=30, kde=True, color='green')
plt.title("Phân bố độ tuổi (Age)")
plt.xlabel("Tuổi")
plt.ylabel("Số lượng")

# Biểu đồ 2: Boxplot cho Fare
plt.subplot(1, 2, 2)
sns.boxplot(x=df['Fare'], color='orange')
plt.title("Phân bố giá vé (Fare)")
plt.xlabel("Giá vé")

plt.tight_layout()
plt.show()

## 4. Xử lý dữ liệu thiếu
Cột `Age` đóng vai trò quan trọng nhưng bị thiếu nhiều dữ liệu. Chúng ta sẽ thay thế các giá trị `NaN` bằng **giá trị trung vị (Median)** để tránh ảnh hưởng bởi các giá trị ngoại lai.

In [ ]:
# Điền giá trị trung vị
median_age = df['Age'].median()
df['Age'].fillna(median_age, inplace=True)

# Kiểm tra lại
print(f"Giá trị trung vị được dùng để điền: {median_age}")
print(f"Số giá trị thiếu của cột 'Age' sau khi xử lý: {df['Age'].isnull().sum()}")

## 5. Chuẩn hóa đặc trưng (Min-Max Scaling)
Đưa các cột số `Age` và `Fare` về cùng một thang đo đoạn `[0, 1]`. Việc này giúp các thuật toán máy học (như KNN, Neural Networks) hoạt động hiệu quả hơn.

In [ ]:
scaler = MinMaxScaler()

# Tạo các cột đã chuẩn hóa
df[['Age_norm', 'Fare_norm']] = scaler.fit_transform(df[['Age', 'Fare']])

# Hiển thị so sánh
print("Dữ liệu trước và sau khi chuẩn hóa:")
display(df[['Age', 'Age_norm', 'Fare', 'Fare_norm']].head())

## 6. Mã hóa dữ liệu phân loại
Máy tính chỉ hiểu các con số, do đó ta cần chuyển đổi dữ liệu chữ:
* **Label Encoding:** Dùng cho cột `Sex` (male=0, female=1).
* **One-Hot Encoding:** Dùng cho cột `Embarked` (Tạo các cột `Embarked_C`, `Embarked_Q`, `Embarked_S`).

In [ ]:
# Label Encoding cho Sex
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

# One-Hot Encoding cho Embarked
df = pd.get_dummies(df, columns=['Embarked'], prefix='Embarked')

# Chuyển đổi True/False sang 0/1 (để nhìn gọn hơn)
cols_to_convert = ['Embarked_C', 'Embarked_Q', 'Embarked_S']
# Kiểm tra xem cột có tồn tại không trước khi convert (phòng trường hợp dữ liệu thiếu Embarked chưa được xử lý triệt để)
existing_cols = [col for col in cols_to_convert if col in df.columns]
df[existing_cols] = df[existing_cols].astype(int)

display(df.head())

## 7. Lưu kết quả
Dữ liệu sạch được lưu vào file `titanic_clean.csv` để sẵn sàng cho huấn luyện mô hình.

In [ ]:
output_path = "../data/titanic_clean.csv"
df.to_csv(output_path, index=False)

print(f"Đã lưu thành công file tại: {output_path}")